## Cycle GAN in PyTorch

In [2]:
%load_ext autoreload
%matplotlib inline

In [10]:
%autoreload 2

from IPython import display

from utils import Logger

import torch

from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

In [11]:
DATA_FOLDER = './torch_data/CycleGAN/MNIST'

In [110]:
import os
import urllib, zipfile
from tqdm import tqdm
import zipfile

VALID_DATA_NAMES = ["ae_photos", "apple2orange", "summer2winter_yosemite", "horse2zebra", "monet2photo", "cezanne2photo", "ukiyoe2photo", "vangogh2photo", "maps", "cityscapes","facades","iphone2dslr_flower","mini", "mini_pix2pix", "mini_colorization"]

URL = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/{}.zip'
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def rename_images(rootdir):
    for subdir, dirs, files in os.walk(rootdir):
        idx = 0
        for file in files:
            file_path = os.path.join(subdir, file)
            new_file_path = os.path.join(subdir, "{}.jpg".format(idx))
            os.rename(file_path, new_file_path)
            idx = idx + 1
        
def download_cyclegan_dataset(filename, path, force=False):
    # Validate dataset filename is valid.
    assert(filename in VALID_DATA_NAMES)
    # Return if path exists.
    file_path = "{}/{}.zip".format(path, filename)
    if(os.path.exists(file_path) and not force): return
    # Otherwise download.
    
    # Make path directory if missing.
    os.makedirs(path, exist_ok=True)
    # Download data
    url = URL.format(filename)
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, file_path, reporthook=t.update_to)
        with zipfile.ZipFile(file_path, 'r') as zip_obj:
            zip_obj.extractall(path)
    rename_images(os.path.join(path, filename))

In [111]:
for name in VALID_DATA_NAMES:
    download_cyclegan_dataset(name, DATA_FOLDER, force=True)

ae_photos.zip: 10.7MB [00:04, 2.47MB/s]                            
apple2orange.zip: 78.5MB [00:13, 5.75MB/s]                              
summer2winter_yosemite.zip: 133MB [01:02, 2.13MB/s]                              
horse2zebra.zip:   8%|▊         | 9.88M/117M [00:16<02:56, 608kB/s]   


KeyboardInterrupt: 

In [158]:
from torch.utils.data import Dataset
from skimage import io

class MyDataset(Dataset):
    """My dataset."""
    
    def __init__(self, path):
        self._path = path
        self.num_files = os.listdir(self._path)

    def __len__(self):
        return self.num_files
    
    def filename(self, idx):
        for directory in ["trainA", "testA"]:
            possible_path = os.path.join(self._path, directory, "{}.jpg".format(str(idx)))
            print(possible_path)
            if os.path.exists(possible_path):
                return possible_path

    def __getitem__(self, idx):
        # Handle vectors.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.filename(idx)
        image = io.imread(img_name)
        sample = {'image': image}

In [159]:
x = MyDataset(DATA_FOLDER + "/ae_photos")

In [160]:
x[100]

./torch_data/CycleGAN/MNIST/ae_photos/trainA/100.jpg
./torch_data/CycleGAN/MNIST/ae_photos/testA/100.jpg


OSError: Cannot understand given URI: None.